In [4]:
import pandas as pd
import numpy as np
import feedparser
from bs4 import BeautifulSoup
import requests
import PyPDF2 as pdf
import io
import os
import re
from gsheet import create
from gsheet import update
import scrape_webpage
import scrape_pdf
import weblist

## Step 1: Scrape the list of webpages

In [5]:
new_orleans_url = "https://www.mvn.usace.army.mil/Missions/Regulatory/Public-Notices/"
web_list = weblist.get_web_list(new_orleans_url)
web_list

,web_link,web_title,published_date,web_expire_date,pdf_url
0,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2015-01960-CM,9/4/2023,9/25/2023,https://www.sam.usace.army.mil//Portals/56/doc...
1,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2014-02629-CM,9/4/2023,10/3/2023,https://www.sam.usace.army.mil//Portals/56/doc...
2,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2019-01224-WPP,9/4/2023,9/25/2023,https://www.sam.usace.army.mil//Portals/56/doc...
3,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2007-00574-EG,8/28/2023,9/27/2023,https://www.sam.usace.army.mil//Portals/56/doc...
4,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2022-00799-CF,8/21/2023,9/11/2023,https://www.sam.usace.army.mil//Portals/56/doc...
...,...,...,...,...,...
2365,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2008-3113-CL,3/4/2013,4/2/2013,https://www.sam.usace.army.mil/http://www.mvn....
2366,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-MCM 2013,3/1/2013,3/31/2013,https://www.sam.usace.army.mil/http://www.mvn....
2367,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-1997-1616-EOO,2/25/2013,3/26/2013,https://www.sam.usace.army.mil/http://www.mvn....
2368,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2012-2234-CL,2/25/2013,3/24/2013,https://www.sam.usace.army.mil/http://www.mvn....


In [9]:
web_list.to_csv("new_orleans_weblist.csv")

## Step2: Scrape the webpage

In [6]:
web_list_sub = web_list.loc[:9]

In [7]:
web_add_df = pd.DataFrame([scrape_webpage.web_extraction(x, 0) for x in web_list_sub["web_link"]],
                            columns = ['web_applicant_contents', 
                                       'web_location', 
                                       'web_character', 
                                       'web_mitigation'])
# Merge with RSS feed table
web_df = web_list_sub.join(web_add_df)

# Add a new column to track the number of errors for each webpage
#rss_web_df["error"] = rss_web_df.apply(lambda row: row.str.contains("ERROR").sum(), axis = 1)

web_df

,web_link,web_title,published_date,web_expire_date,pdf_url,web_applicant_contents,web_location,web_character,web_mitigation
0,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2015-01960-CM,9/4/2023,9/25/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"Ascension Pipeline LLC, c/o SWCA Environmental...",Beginning at the Marathon Refinery in Garyvill...,A permit for the work described herein was pre...,None
1,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2014-02629-CM,9/4/2023,10/3/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"The Mosaic Company, c/o Matrix PDM, Attn: Bill...","In the Mississippi River, LDB, River Mile 160....",The applicant has requested Department of the ...,None
2,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2019-01224-WPP,9/4/2023,9/25/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"Terrebonne Parish Consolidated Government, c/o...","Along Lake Chien and Lake Tambour, in Terrebon...",Construct approximately 3.54 linear miles of o...,None
3,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2007-00574-EG,8/28/2023,9/27/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"Cooper Consolidated, c/o Gulf South Engineerin...","At approximately Latitude: 30.046667, Longitud...",Expand an existing barge fleet to add one (1) ...,None
4,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2022-00799-CF,8/21/2023,9/11/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"Scott Thomas Development, LLC c/o: Trusted Com...","Located in Brusly, Louisiana, within West Bato...",The applicant has requested Department of the ...,None
5,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2023-00140-ECS,8/21/2023,9/11/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"St. Charles Parish Government, c/o: Civix, Ms....","Located in Section 20, Township 15 South, Rang...",The proposed project is to widen the existing ...,None
6,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2022-00476-ECS,8/21/2023,9/11/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"STOA Group, c/o: Hydrik Wetlands Consultants, ...","Located northerly of McKinney Road, Slidell, i...","The proposed project includes the clearing, gr...",None
7,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2022-01256-WPP,8/21/2023,9/11/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"Morganfield South, LLC, c/o: Rhett Francois, 1...","Located in Calcasieu Parish, near the communit...","Clear, grade, excavate and fill (approximately...",None
8,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-1999-02360-EPP,8/14/2023,9/4/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"English Turn Limited Partnership, Attention: J...","At approximately Latitude: 29.90826, Longitude...",Continual operations to install and maintain f...,None
9,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2020-00234-WPP,8/14/2023,9/4/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"Port of Iberia, c/o: GIS Engineering, LLC, 450...","Located in Iberia Parish, Section 34, Township...",A Department of Army permit was issued on Apri...,None


## Step3: Scrap PDFs

In [8]:
pdf_df = pd.DataFrame([scrape_pdf.pdf_extraction(x) for x in web_df["pdf_url"]],
                            columns = ['pdf_permit_app_num',
                                       'pdf_dist_code',
                                       'pdf_dist_name',  
                                       'pdf_manager_name', 
                                       'pdf_manager_phone', 
                                       'pdf_manager_email',
                                       'pdf_applicant_contents',
                                       'pdf_applicant',
                                       'pdf_contract',
                                       'pdf_location', 
                                       'pdf_character', 
                                       'pdf_county',
                                       'pdf_parish',
                                       'pdf_hydrologic_unit_code', 
                                       'pdf_longitude',
                                       'pdf_latitude',
                                       'pdf_acreage',
                                       'pdf_wqc',
                                       'pdf_cup'])

# # Merge with RSS feed table
final_df = web_df.join(pdf_df)

# Add a new column to track the number of errors for each webpage
final_df["error"] = final_df.apply(lambda row: row.str.contains("ERROR").sum(), axis = 1)

final_df

,web_link,web_title,published_date,web_expire_date,pdf_url,web_applicant_contents,web_location,web_character,web_mitigation,pdf_permit_app_num,...,pdf_character,pdf_county,pdf_parish,pdf_hydrologic_unit_code,pdf_longitude,pdf_latitude,pdf_acreage,pdf_wqc,pdf_cup,error
0,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2015-01960-CM,9/4/2023,9/25/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"Ascension Pipeline LLC, c/o SWCA Environmental...",Beginning at the Marathon Refinery in Garyvill...,A permit for the work described herein was pre...,None,ERROR: 'Num',...,A permit for the work descr ibed herein was pr...,None,"Geismar, Louisiana. The project will span Asce...",None,None,None,"[7.169, 5.755]",None,None,3
1,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2014-02629-CM,9/4/2023,10/3/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"The Mosaic Company, c/o Matrix PDM, Attn: Bill...","In the Mississippi River, LDB, River Mile 160....",The applicant has requested Department of the ...,None,ERROR: 'Num',...,The applicant has requested Department of the ...,None,St. James,None,None,None,[1.5],None,None,3
2,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2019-01224-WPP,9/4/2023,9/25/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"Terrebonne Parish Consolidated Government, c/o...","Along Lake Chien and Lake Tambour, in Terrebon...",Construct approximately 3.54 linear miles of o...,None,MVN-2019-01224-WPP,...,Construct approximately 3.54 linear miles of o...,None,Terre bonne,080903 02,[90.44004],[29.3488],[6.6],None,None,0
3,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2007-00574-EG,8/28/2023,9/27/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"Cooper Consolidated, c/o Gulf South Engineerin...","At approximately Latitude: 30.046667, Longitud...",Expand an existing barge fleet to add one (1) ...,None,MVN-2007-00574-EG,...,Expand an existing barge fleet to add one (1) ...,None,"LaPlace, Louisiana, in St. John the Baptist",08070100,[90.474722],[],None,None,None,0
4,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2022-00799-CF,8/21/2023,9/11/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"Scott Thomas Development, LLC c/o: Trusted Com...","Located in Brusly, Louisiana, within West Bato...",The applicant has requested Department of the ...,None,MVN-2022-00799-CF,...,The applicant has requested Department of the ...,None,"Brusly, Louisiana, within West Baton Rouge",08070300,[91.317175],[30.393175],[7.43],None,None,0
5,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2023-00140-ECS,8/21/2023,9/11/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"St. Charles Parish Government, c/o: Civix, Ms....","Located in Section 20, Township 15 South, Rang...",The proposed project is to widen the existing ...,None,MVN -2023-00140-ECS,...,The proposed project is to widen the existing...,None,"Section 20, Township 15 South, Range 20 East ,...",08 0903010302,[90.42970],[29.78860],[],None,None,0
6,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2022-00476-ECS,8/21/2023,9/11/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"STOA Group, c/o: Hydrik Wetlands Consultants, ...","Located northerly of McKinney Road, Slidell, i...","The proposed project includes the clearing, gr...",None,MVN -2022 -00476 -ECS,...,"The proposed project includes the clearing , g...",None,St. Tammany,08090 2010 307,[89.757675],[30.275742],[],None,None,0
7,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2022-01256-WPP,8/21/2023,9/11/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"Morganfield South, LLC, c/o: Rhett Francois, 1...","Located in Calcasieu Parish, near the communit...","Clear, grade, excavate and fill (approximately...",None,MVN-2022-01256-WPP,...,"Clear, grade, excavate and fill (approximately...",None,Calcasieu,08080206,[93.161006],[30.16603],[],None,None,0
8,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-1999-02360-EPP,8/14/2023,9/4/2023,https://www.sam.usa